In [1]:
%load_ext autoreload
%autoreload 2

In [10]:
import pandas as pd
import numpy as np

# Data Prep

## Create mutated sequences

In [11]:
df = pd.read_csv('train.csv')

# get the sequence
seq = open('sequence.fasta', 'r').read()
seq = seq.split("\n")[1]

# create each mutated sequence using the info
sequences = []
for i in df['mutant']:
    ind = int(i[1:-1])
    tmp = seq[:ind] + i[-1] + seq[ind+1:]
    sequences.append(tmp)
df['sequence'] = sequences

In [20]:
SEQ_LENGTH = len(seq)

## Prepare One_hot

In [12]:
# Define the amino acids
amino_acids = "ACDEFGHIKLMNPQRSTVWXY"  # 20 standard amino acids

# Create a dictionary to map amino acids to indices
aa_dict = {aa: idx for idx, aa in enumerate(amino_acids)}

# Function to convert sequence to one-hot encoding
def seq_to_one_hot(sequence):
    one_hot = np.zeros((len(sequence), len(amino_acids)))
    for i, aa in enumerate(sequence):
        if aa in aa_dict:
            one_hot[i, aa_dict[aa]] = 1
    return one_hot

In [13]:
# Convert the sequences into one-hot encodings
df['one_hot_sequence'] = df['sequence'].apply(lambda seq: seq_to_one_hot(seq).tolist())


In [14]:
df.head()

,mutant,DMS_score,sequence,one_hot_sequence
0,M0Y,0.2730,YVNEARGNSSLNPCLEGSASSGSESSKDSSRCSTPGLDPERHERLR...,"[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,..."
1,M0W,0.2857,WVNEARGNSSLNPCLEGSASSGSESSKDSSRCSTPGLDPERHERLR...,"[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,..."
2,M0V,0.2153,VVNEARGNSSLNPCLEGSASSGSESSKDSSRCSTPGLDPERHERLR...,"[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,..."
3,M0T,0.3122,TVNEARGNSSLNPCLEGSASSGSESSKDSSRCSTPGLDPERHERLR...,"[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,..."
4,M0S,0.2180,SVNEARGNSSLNPCLEGSASSGSESSKDSSRCSTPGLDPERHERLR...,"[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,..."


## Prepare K_validation

In [51]:
# Prepare the data
def get_data(df):
    X = np.array(df['one_hot_sequence'].tolist())  # One-hot encoded sequences
    X = X.transpose(0, 2, 1) # shape expected (N, C_in, L)
    y = np.array(df['DMS_score'].tolist())  # DMS scores (target)
    return torch.tensor(X, dtype=torch.float32), torch.tensor(y, dtype=torch.float32)


# CNN Model

## Model Def

In [56]:
# Define the CNN model for input shape (batch_size, 656, 21)
class AminoAcidCNN(nn.Module):
    def __init__(self, seq_length, num_amino_acids):
        super(AminoAcidCNN, self).__init__()

        # Convolutional layers
        self.conv1 = nn.Conv1d(in_channels=num_amino_acids, out_channels=32, kernel_size=3, padding=1)  # 32 filters
        self.conv2 = nn.Conv1d(in_channels=32, out_channels=64, kernel_size=3, padding=1)  # 64 filters
        self.conv3 = nn.Conv1d(in_channels=64, out_channels=128, kernel_size=3, padding=1)  # 128 filters
        self.pool = nn.MaxPool1d(kernel_size=2, stride=2)

        # Fully connected layers
        # After applying 3 convolution layers and pooling, the sequence length will be reduced by a factor of 8
        self.fc1 = nn.Linear(128 * (seq_length // 8), 64)  # Adjust the output dimension based on the input length
        self.fc2 = nn.Linear(64, 1)  # Single output for fitness score

    def forward(self, x):
        # Forward pass through convolutional layers
        x = self.pool(torch.relu(self.conv1(x)))
        x = self.pool(torch.relu(self.conv2(x)))
        x = self.pool(torch.relu(self.conv3(x)))

        # Flatten the tensor for the fully connected layers
        x = x.view(x.size(0), -1)  # Flatten

        # Fully connected layers
        x = torch.relu(self.fc1(x))
        x = self.fc2(x)  # Output layer (fitness score)
        return x

# Example usage:
num_amino_acids = len(amino_acids)  # For standard amino acids (A, C, D, E, F, G, H, I, K, L, M, N, P, Q, R, S, T, V, W, Y)

# Create the model
cnn_model = AminoAcidCNN(SEQ_LENGTH, num_amino_acids)

# Print the model summary (optional)
print(cnn_model)

# Example input: batch size of 32, sequence length 100, and 20


AminoAcidCNN(
  (conv1): Conv1d(21, 32, kernel_size=(3,), stride=(1,), padding=(1,))
  (conv2): Conv1d(32, 64, kernel_size=(3,), stride=(1,), padding=(1,))
  (conv3): Conv1d(64, 128, kernel_size=(3,), stride=(1,), padding=(1,))
  (pool): MaxPool1d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (fc1): Linear(in_features=10496, out_features=64, bias=True)
  (fc2): Linear(in_features=64, out_features=1, bias=True)
)


## Training

In [57]:
from sklearn.model_selection import KFold

K = 5
EPOCHS = 100
BATCH_SIZE = 32

In [58]:
X, y = get_data(df)
num_samples, seq_length, num_amino_acids = X.shape

kfold = KFold(n_splits=K, shuffle=True, random_state=42)

In [64]:

fold_accuracies = []
for fold, (train_idx, val_idx) in enumerate(kfold.split(X)):
    print(f"Training fold {fold + 1}/{k}")
    
    # Split the data into training and validation sets
    X_train, X_val = X[train_idx], X[val_idx]
    y_train, y_val = y[train_idx], y[val_idx]
    
    # Initialize the model
    model = AminoAcidCNN(SEQ_LENGTH, len(amino_acids))
    
    # Define the optimizer and loss function
    optimizer = optim.Adam(model.parameters(), lr=5e-4)
    criterion = nn.MSELoss()  # Mean Squared Error for regression

    # Training the model
    for epoch in range(EPOCHS):
        model.train()
        optimizer.zero_grad()
        
        # Create batches for training
        for i in range(0, len(X_train), BATCH_SIZE):
            X_batch = X_train[i:i + BATCH_SIZE]
            y_batch = y_train[i:i + BATCH_SIZE]

            # Forward pass
            outputs = model(X_batch)
            loss = criterion(outputs, y_batch.view(-1, 1))

            # Backward pass and optimization
            loss.backward()
            optimizer.step()

        # Print loss for each epoch
        print(f"Epoch [{epoch+1}/{EPOCHS}], Loss: {loss.item():.4f}")
    
    # Evaluate the model
    model.eval()
    with torch.no_grad():
        val_outputs = model(X_val)
        val_loss = criterion(val_outputs, y_val.view(-1, 1))
        fold_accuracies.append(val_loss.item())
        print(f"Validation Loss for fold {fold + 1}: {val_loss.item():.4f}")

# Average validation loss across all folds
avg_loss = np.mean(fold_accuracies)
print(f"Average Validation Loss across {k} folds: {avg_loss:.4f}")


Training fold 1/5
Epoch [1/100], Loss: 0.2884
Epoch [2/100], Loss: 0.0434
Epoch [3/100], Loss: 0.1611
Epoch [4/100], Loss: 0.0376
Epoch [5/100], Loss: 0.2664
Epoch [6/100], Loss: 0.0944
Epoch [7/100], Loss: 0.1452
Epoch [8/100], Loss: 0.1012
Epoch [9/100], Loss: 0.0611
Epoch [10/100], Loss: 0.0726
Epoch [11/100], Loss: 0.0682
Epoch [12/100], Loss: 0.0673
Epoch [13/100], Loss: 0.0730
Epoch [14/100], Loss: 0.0629
Epoch [15/100], Loss: 0.0829
Epoch [16/100], Loss: 0.0600
Epoch [17/100], Loss: 0.0972
Epoch [18/100], Loss: 0.0810
Epoch [19/100], Loss: 0.0679
Epoch [20/100], Loss: 0.0752
Epoch [21/100], Loss: 0.0660
Epoch [22/100], Loss: 0.0708
Epoch [23/100], Loss: 0.0678
Epoch [24/100], Loss: 0.0689
Epoch [25/100], Loss: 0.0690
Epoch [26/100], Loss: 0.0678
Epoch [27/100], Loss: 0.0702
Epoch [28/100], Loss: 0.0666
Epoch [29/100], Loss: 0.0725
Epoch [30/100], Loss: 0.0644
Epoch [31/100], Loss: 0.0786
Epoch [32/100], Loss: 0.0619
Epoch [33/100], Loss: 0.0914
Epoch [34/100], Loss: 0.0709
Epoch

## Test Performance

In [65]:
from scipy.stats import spearmanr

def compute_predictions_and_spearman(df, model):
    # Get the data from the dataframe
    X, y = get_data(df)

    # Ensure the model is in evaluation mode
    model.eval()

    # Compute predictions
    with torch.no_grad():
        predicted_dms_scores = model(X).squeeze().numpy()  # Flatten to 1D array

    # Compute Spearman correlation
    correlation, _ = spearmanr(predicted_dms_scores, y.numpy())  # Compute Spearman correlation between actual and predicted DMS scores

    return predicted_dms_scores, correlation

_, correlation = compute_predictions_and_spearman(df, model)
correlation

0.30486666780170124